In [2]:
import edgar
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup as bsoup
import numpy as np
import string
from cleanMDA import extractTable, getComp, divide_chunks, annual_filings, MDA, pullMDA
from collections import defaultdict

The following code gathers the data needed for our analysis. We first finds the top 10 companies in the S&P 500 based on total market capitalization for each year of analysis, 2007 - 2017. This data is web scrapped from two data sources, ETF database and Siblis Research. Next, we match the top ten companies to their respective CIK number, which is used to lookup companies on the SEC website (Refer to the dataframe dfTop10). 

Given this data, we web scrapped the SEC edgar website (https://www.sec.gov/) for each companies MD&A section of the 10-K annual report. Refer to functions MDA, annual_fillings and pullMDA in the CleanMDA.py file. 

**Get Top 2007-2012 Companies**

In [3]:
res = requests.get('https://etfdb.com/history-of-the-s-and-p-500/#2009')
soup = bsoup(res.content,'html.parser')
tbl = extractTable(soup)

In [4]:
# Remove companies 1980-1999
Lst = tbl[200:]
# Split into chunks of 10 CO per year 
tbl = list(divide_chunks(Lst, 10))
colNames=[i for i in range(2000,2014)]
# Make df
dfTop10 = pd.DataFrame(tbl).transpose()
dfTop10.columns = colNames

In [5]:
# Remove punctuation and uppercase 
for i in colNames:
    dfTop10[i] = dfTop10[i].str.upper().str.strip(string.punctuation).str.strip('\n').str.strip()


In [6]:
dfTop10.replace({'CHEVRON CORP': 'CHEVRON', 'CHEVRON': 'CHEVRON CORP','MICROSOFT': 'MICROSOFT CORP','GENERAL ELECTRIC': 'GENERAL ELECTRIC CO', 'EXXON MOBIL':'EXXON MOBIL CORP', 'PFIZER':'PFIZER INC', 'CITIGROUP':'CITIGROUP INC', 'CISCO SYSTEMS':'CISCO SYSTEMS INC', 'WAL-MART STORES' :'WALMART INC','AIG':'AMERICAN INTERNATIONAL GROUP','MERCK':'MERCK & CO','INTEL':'INTEL CORP','PROCTER & GAMBLE':'PROCTER & GAMBLE CO','AT&T':'AT&T CORP','JPMORGAN CHASE':'JPMORGAN CHASE & CO','GOOGLE':'ALPHABET INC','APPLE':'APPLE INC','BANK OF AMERICA':'BANK OF AMERICA CORP','IBM':'INTL BUSINESS MACHINES CORP','CHEVRON':'CHEVRON CORP','WELLS FARGO':'WELLS FARGO & CO'}, regex=True, inplace=True)

In [7]:
dfCik = pd.read_csv('2000_2013_s&p.csv')
dfCik.drop(['gvkey','gvkeyx','from','thru','tic'], axis=1, inplace=True)
dfCik['co_cik'] = dfCik['co_cik'].fillna(0).astype(int).apply(lambda x: '{0:0>10}'.format(x))
cikDic = pd.Series(dfCik.co_conm.values,index=dfCik.co_cik).to_dict()
cikDic = dict((v,k) for k,v in cikDic.items())

In [8]:
from collections import defaultdict
dic = defaultdict(None)
# Create a dictionary of Company Name and CIK #
for yr in colNames:  
    for comp in dfTop10[yr]:
        if comp in cikDic:
            dic[comp] = cikDic[comp]
        else:
            dic[comp] = None
        

In [9]:
# Create Company Name, CIK number tuple pairs 
for yr in colNames:  
    
    for i in range(len(dfTop10[yr])):
        if type(dfTop10[yr][i])==tuple:
            pass
        else:
            name = dfTop10[yr][i]
            dfTop10[yr][i] =  (name,dic[name])

In [10]:
dfTop10 = dfTop10.drop(columns=[2000,2001,2002,2003,2004,2005,2006,2013])

In [11]:
# Clean wrong CIK numbers 
for year in dfTop10.columns:
    for cmp in range(10):
        if dfTop10[year][cmp] == ('ALPHABET INC', '0001652044'):
            dfTop10[year].loc[cmp] = ('ALPHABET INC', '0001288776')
        elif dfTop10[year][cmp] == ('AT&T CORP', '0000005907'):
            dfTop10[year].loc[cmp] = ('AT&T CORP', '0000732717')

In [12]:
# reorder the yeara
dfTop10 = dfTop10[dfTop10.columns[::-1]]

**Get 2013-2017 Companies**

In [13]:
# read in top 100 2013-2017 companies
df = pd.read_html('http://siblisresearch.com/data/market-caps-sp-100-us/')[0]

In [14]:
# get top 10 companies from each year
df_2017 = pd.DataFrame(df['Company Name'])
df_2017['12/31/2017'] = df['12/31/2017']
df_2017 = df_2017.sort_values(by=['12/31/2017'], ascending=False)
top_2017 = pd.DataFrame(np.array(df_2017.head(10)['Company Name']), columns=['2017'])

df_2016 = pd.DataFrame(df['Company Name'])
df_2016['12/31/2016'] = df['12/31/2016']
df_2016 = df_2016.sort_values(by=['12/31/2016'], ascending=False)
top_2016 = pd.DataFrame(np.array(df_2016.head(10)['Company Name']), columns=['2016'])

df_2015 = pd.DataFrame(df['Company Name'])
df_2015['12/31/2015'] = df['12/31/2015']
df_2015 = df_2015.sort_values(by=['12/31/2015'], ascending=False)
top_2015 = pd.DataFrame(np.array(df_2015.head(10)['Company Name']), columns=['2015'])

df_2014 = pd.DataFrame(df['Company Name'])
df_2014['12/31/2014'] = df['12/31/2014']
df_2014 = df_2014.sort_values(by=['12/31/2014'], ascending=False)
top_2014 = pd.DataFrame(np.array(df_2014.head(10)['Company Name']), columns=['2014'])

df_2013 = pd.DataFrame(df['Company Name'])
df_2013['12/31/2013'] = df['12/31/2013']
df_2013 = df_2013.sort_values(by=['12/31/2013'], ascending=False)
top_2013 = pd.DataFrame(np.array(df_2013.head(10)['Company Name']), columns=['2013'])

In [15]:
top10_2017 = pd.concat([top_2017,top_2016,top_2015,top_2014,top_2013],axis=1)

In [16]:
# get an array of the Company names and CIK numbers
companies = pd.DataFrame.from_csv('companies_2017.csv',encoding='latin-1')
companies = companies[:505]
companies.CIK = companies['CIK'].astype(int).apply(lambda x: str(x).zfill(10))

name = [i.strip(string.punctuation) for i in np.array(companies['Security'])]
num = np.array(companies['CIK'])

# get companies CIK numbers
CIK_num = dict()
for i in range(len(name)):
    CIK_num[name[i]] = num[i]

# manually add in ID for Company Name's missed
CIK_man = {'Alphabet Inc. Class A':'0001652044','Berkshire Hathaway Inc':'0001067983', 'Wal-Mart Stores Inc':'0000104169', 
           'IBM Corp.':'0000051143', 'The Coca Cola Co.':'0000021344', "Philip Morris Int'l":'0001413329', 
           'Facebook Inc.':'0001326801', 'The Walt Disney Co.':'0001001039', 'Comcast Class A Comm.':'0001166691'}

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  


In [17]:
# returns an array of each companies CIK number
for year in top10_2017:
    for i in range(len(top10_2017[year])):
        if top10_2017[year][i].strip(string.punctuation) in CIK_num:
            top10_2017[year][i] = (top10_2017[year][i],CIK_num[top10_2017[year][i].strip(string.punctuation)])
        elif int(year) < 2016 and top10_2017[year][i] == 'Alphabet Inc. Class A':
            top10_2017[year][i] = (top10_2017[year][i],'0001288776')
        elif top10_2017[year][i] in CIK_man:
            top10_2017[year][i] = (top10_2017[year][i],CIK_man[top10_2017[year][i]])
        else:
            top10_2017[year][i] = (top10_2017[year][i], None)
top10_2017.head()

,2017,2016,2015,2014,2013
0,"(Apple Inc., 0000320193)","(Apple Inc., 0000320193)","(Apple Inc., 0000320193)","(Apple Inc., 0000320193)","(Apple Inc., 0000320193)"
1,"(Alphabet Inc. Class A, 0001652044)","(Alphabet Inc. Class A, 0001652044)","(Alphabet Inc. Class A, 0001288776)","(Exxon Mobil Corp., 0000034088)","(Exxon Mobil Corp., 0000034088)"
2,"(Microsoft Corp., 0000789019)","(Microsoft Corp., 0000789019)","(Microsoft Corp., 0000789019)","(Microsoft Corp., 0000789019)","(Alphabet Inc. Class A, 0001288776)"
3,"(Amazon.com Inc., 0001018724)","(Berkshire Hathaway Inc, 0001067983)","(Berkshire Hathaway Inc, 0001067983)","(Berkshire Hathaway Inc, 0001067983)","(Microsoft Corp., 0000789019)"
4,"(Facebook Inc., 0001326801)","(Exxon Mobil Corp., 0000034088)","(Exxon Mobil Corp., 0000034088)","(Alphabet Inc. Class A, 0001288776)","(Berkshire Hathaway Inc, 0001067983)"


In [18]:
# combine all the years together
dfTop10 = pd.concat([top10_2017,dfTop10],axis=1)
# change all the years to integers
dfTop10 = pd.DataFrame(np.array(dfTop10), columns=dfTop10.columns.astype(int))
dfTop10.head(3)

,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008,2007
0,"(Apple Inc., 0000320193)","(Apple Inc., 0000320193)","(Apple Inc., 0000320193)","(Apple Inc., 0000320193)","(Apple Inc., 0000320193)","(APPLE INC, 0000320193)","(EXXON MOBIL CORP, 0000034088)","(EXXON MOBIL CORP, 0000034088)","(EXXON MOBIL CORP, 0000034088)","(EXXON MOBIL CORP, 0000034088)","(EXXON MOBIL CORP, 0000034088)"
1,"(Alphabet Inc. Class A, 0001652044)","(Alphabet Inc. Class A, 0001652044)","(Alphabet Inc. Class A, 0001288776)","(Exxon Mobil Corp., 0000034088)","(Exxon Mobil Corp., 0000034088)","(EXXON MOBIL CORP, 0000034088)","(APPLE INC, 0000320193)","(APPLE INC, 0000320193)","(MICROSOFT CORP, 0000789019)","(WALMART INC, 0000104169)","(GENERAL ELECTRIC CO, 0000040545)"
2,"(Microsoft Corp., 0000789019)","(Microsoft Corp., 0000789019)","(Microsoft Corp., 0000789019)","(Microsoft Corp., 0000789019)","(Alphabet Inc. Class A, 0001288776)","(MICROSOFT CORP, 0000789019)","(MICROSOFT CORP, 0000789019)","(MICROSOFT CORP, 0000789019)","(WALMART INC, 0000104169)","(PROCTER & GAMBLE CO, 0000080424)","(MICROSOFT CORP, 0000789019)"


**Find the indexes of each company's MDA text**

In [19]:
# create a dataframe to hold the indexes and document number
dfTop10_index = dfTop10.copy()

In [20]:
dfTop10_index.head(3)

,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008,2007
0,"(Apple Inc., 0000320193)","(Apple Inc., 0000320193)","(Apple Inc., 0000320193)","(Apple Inc., 0000320193)","(Apple Inc., 0000320193)","(APPLE INC, 0000320193)","(EXXON MOBIL CORP, 0000034088)","(EXXON MOBIL CORP, 0000034088)","(EXXON MOBIL CORP, 0000034088)","(EXXON MOBIL CORP, 0000034088)","(EXXON MOBIL CORP, 0000034088)"
1,"(Alphabet Inc. Class A, 0001652044)","(Alphabet Inc. Class A, 0001652044)","(Alphabet Inc. Class A, 0001288776)","(Exxon Mobil Corp., 0000034088)","(Exxon Mobil Corp., 0000034088)","(EXXON MOBIL CORP, 0000034088)","(APPLE INC, 0000320193)","(APPLE INC, 0000320193)","(MICROSOFT CORP, 0000789019)","(WALMART INC, 0000104169)","(GENERAL ELECTRIC CO, 0000040545)"
2,"(Microsoft Corp., 0000789019)","(Microsoft Corp., 0000789019)","(Microsoft Corp., 0000789019)","(Microsoft Corp., 0000789019)","(Alphabet Inc. Class A, 0001288776)","(MICROSOFT CORP, 0000789019)","(MICROSOFT CORP, 0000789019)","(MICROSOFT CORP, 0000789019)","(WALMART INC, 0000104169)","(PROCTER & GAMBLE CO, 0000080424)","(MICROSOFT CORP, 0000789019)"


Note that the below code takes some time to run as it pulls the indices for MDA for all years. The final file is pickled below.

In [21]:
# go through each company and get the indices of which section to pull the MDA from
ct = 0
for year in dfTop10:
    ct += 1
    for i in range(len(dfTop10[year])):
        dfTop10_index[year].iloc[i] = annual_filings(dfTop10[year].iloc[i][0],dfTop10[year].iloc[i][1],year,ct)
dfTop10_index.head()

,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008,2007
0,"(Apple Inc., 0000320193, [10, 32, 41, 43], [14...","(Apple Inc., 0000320193, [10, 31, 40, 42], [14...","(Apple Inc., 0000320193, [10, 31, 40, 42], [14...","(Apple Inc., 0000320193, [10, 31, 40, 42], [14...","(Apple Inc., 0000320193, [10, 30, 39, 41, 42],...","(APPLE INC, 0000320193, [17, 19, 20], [36], 6)","(EXXON MOBIL CORP, 0000034088, [11, 35], [15, ...","(EXXON MOBIL CORP, 0000034088, [10, 33], [14, ...","(EXXON MOBIL CORP, 0000034088, [11, 35], [15, ...","(EXXON MOBIL CORP, 0000034088, [11, 34], [15, ...","(EXXON MOBIL CORP, 0000034088, [11, 35], [15, ..."
1,"(Alphabet Inc. Class A, 0001652044, [11, 26, 3...","(Alphabet Inc. Class A, 0001652044, [11, 25, 3...","(Alphabet Inc. Class A, 0001288776, [11, 25, 3...","(Exxon Mobil Corp., 0000034088, [11, 35, 52, 5...","(Exxon Mobil Corp., 0000034088, [11, 35, 52, 5...","(EXXON MOBIL CORP, 0000034088, [11, 35, 53, 54...","(APPLE INC, 0000320193, [9, 16, 18], [30], 7)","(APPLE INC, 0000320193, [9, 16, 18], [28], 8)","(MICROSOFT CORP, 0000789019, [10, 53], [14, 12...","(WALMART INC, 0000104169, [17], [24], 11)","(GENERAL ELECTRIC CO, 0000040545, [10, 34], [1..."
2,"(Microsoft Corp., 0000789019, [10, 67], [14, 1...","(Microsoft Corp., 0000789019, [10, 65], [14, 1...","(Microsoft Corp., 0000789019, [10, 64], [14, 1...","(Microsoft Corp., 0000789019, [10, 60], [14, 1...","(Alphabet Inc. Class A, 0001288776, [10, 24, 3...","(MICROSOFT CORP, 0000789019, [10, 57], [14, 12...","(MICROSOFT CORP, 0000789019, [9, 53], [13, 125...","(MICROSOFT CORP, 0000789019, [9, 55], [13, 126...","(WALMART INC, 0000104169, [17], [24], 10)","(PROCTER & GAMBLE CO, 0000080424, [10, 36], [1...","(MICROSOFT CORP, 0000789019, [10, 52], [14, 11..."
3,"(Amazon.com Inc., 0001018724, [10, 46], [14, 6...","(Berkshire Hathaway Inc, 0001067983, [10, 33, ...","(Berkshire Hathaway Inc, 0001067983, [10, 33, ...","(Berkshire Hathaway Inc, 0001067983, [10, 32, ...","(Microsoft Corp., 0000789019, [10, 56], [14, 1...","(GENERAL ELECTRIC CO, 0000040545, [10, 25, 27,...","(INTL BUSINESS MACHINES CORP, 0000051143, [14,...","(BERKSHIRE HATHAWAY, 0001067983, [10, 32, 34],...","(ALPHABET INC, 0001288776, [10, 33], [14, 45], 8)","(MICROSOFT CORP, 0000789019, [10, 52], [14, 11...","(AT&T CORP, 0000732717, [2, 13], [2, 20], 11)"
4,"(Facebook Inc., 0001326801, [10, 24, 33, 36], ...","(Exxon Mobil Corp., 0000034088, [11, 35, 53, 5...","(Exxon Mobil Corp., 0000034088, [11, 35, 53, 5...","(Alphabet Inc. Class A, 0001288776, [10, 24, 3...","(Berkshire Hathaway Inc, 0001067983, [10, 32, ...","(INTL BUSINESS MACHINES CORP, 0000051143, [14,...","(CHEVRON CORP, 0000093410, [2, 15], [2, 19], 7)","(GENERAL ELECTRIC CO, 0000040545, [10, 26, 28,...","(APPLE INC, 0000320193, [9, 17, 19], [30], 10)","(AT&T CORP, 0000732717, [2, 13], [2, 20], 10)","(PROCTER & GAMBLE CO, 0000080424, [10, 36], [1..."


**Find the MDA text for each company**

In [22]:
# create a dataframe to hold the MDA text
dfTop10_MDA = dfTop10_index.copy()
dfTop10_MDA.head(3)

,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008,2007
0,"(Apple Inc., 0000320193, [10, 32, 41, 43], [14...","(Apple Inc., 0000320193, [10, 31, 40, 42], [14...","(Apple Inc., 0000320193, [10, 31, 40, 42], [14...","(Apple Inc., 0000320193, [10, 31, 40, 42], [14...","(Apple Inc., 0000320193, [10, 30, 39, 41, 42],...","(APPLE INC, 0000320193, [17, 19, 20], [36], 6)","(EXXON MOBIL CORP, 0000034088, [11, 35], [15, ...","(EXXON MOBIL CORP, 0000034088, [10, 33], [14, ...","(EXXON MOBIL CORP, 0000034088, [11, 35], [15, ...","(EXXON MOBIL CORP, 0000034088, [11, 34], [15, ...","(EXXON MOBIL CORP, 0000034088, [11, 35], [15, ..."
1,"(Alphabet Inc. Class A, 0001652044, [11, 26, 3...","(Alphabet Inc. Class A, 0001652044, [11, 25, 3...","(Alphabet Inc. Class A, 0001288776, [11, 25, 3...","(Exxon Mobil Corp., 0000034088, [11, 35, 52, 5...","(Exxon Mobil Corp., 0000034088, [11, 35, 52, 5...","(EXXON MOBIL CORP, 0000034088, [11, 35, 53, 54...","(APPLE INC, 0000320193, [9, 16, 18], [30], 7)","(APPLE INC, 0000320193, [9, 16, 18], [28], 8)","(MICROSOFT CORP, 0000789019, [10, 53], [14, 12...","(WALMART INC, 0000104169, [17], [24], 11)","(GENERAL ELECTRIC CO, 0000040545, [10, 34], [1..."
2,"(Microsoft Corp., 0000789019, [10, 67], [14, 1...","(Microsoft Corp., 0000789019, [10, 65], [14, 1...","(Microsoft Corp., 0000789019, [10, 64], [14, 1...","(Microsoft Corp., 0000789019, [10, 60], [14, 1...","(Alphabet Inc. Class A, 0001288776, [10, 24, 3...","(MICROSOFT CORP, 0000789019, [10, 57], [14, 12...","(MICROSOFT CORP, 0000789019, [9, 53], [13, 125...","(MICROSOFT CORP, 0000789019, [9, 55], [13, 126...","(WALMART INC, 0000104169, [17], [24], 10)","(PROCTER & GAMBLE CO, 0000080424, [10, 36], [1...","(MICROSOFT CORP, 0000789019, [10, 52], [14, 11..."


Again note that the following code will take time to run as this pulls the text given the indicies that were pulled in the code above. 

In [23]:
# go through each company and get the indices of which section to pull the MDA from
for year in dfTop10_index:
    for i in range(len(dfTop10_index[year])):
        if dfTop10_index[year][i][1] != None:
            dfTop10_MDA[year].iloc[i] = MDA(dfTop10_index[year].iloc[i][0],dfTop10_index[year].iloc[i][1],dfTop10_index[year].iloc[i][2],dfTop10_index[year].iloc[i][3],dfTop10_index[year].iloc[i][4],year)
        else:
            dfTop10_MDA[year].iloc[i]= dfTop10_index[year][i]

In [24]:
dfTop10_MDA.head(3)

,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008,2007
0,"(Apple Inc., [7 MANAGEMENT’S DISCUSSION AND AN...","(Apple Inc., [7 MANAGEMENT’S DISCUSSION AND AN...","(Apple Inc., [7 MANAGEMENTS DISCUSSION AND AN...","(Apple Inc., [7 MANAGEMENTS DISCUSSION AND AN...","(Apple Inc., [7 MANAGEMENTS DISCUSSION AND A...","(APPLE INC, [7 MANAGEMENTS DISCUSSION AND AN...","(EXXON MOBIL CORP, [15. EXHIBITS, FINANCIAL ST...","(EXXON MOBIL CORP, [15. EXHIBITS, FINANCIAL...","(EXXON MOBIL CORP, [15. EXHIBITS, FINANCIAL...","(EXXON MOBIL CORP, [15. EXHIBITS, FINANCIAL...","(EXXON MOBIL CORP, [15. EXHIBITS, FINANCIAL..."
1,"(Alphabet Inc. Class A, [7MANAGEMENT’S DISCUSS...","(Alphabet Inc. Class A, [7MANAGEMENT’S DISCUSS...","(Alphabet Inc. Class A, [7MANAGEMENT’S DISCUSS...","(Exxon Mobil Corp., [15. EXHIBITS, FINANCI...","(Exxon Mobil Corp., [15. EXHIBITS, FINANCI...","(EXXON MOBIL CORP, [15. EXHIBITS, FINANCIA...","(APPLE INC, [7 MANAGEMENTS DISCUSSION AND ANA...","(APPLE INC, [7 MANAGEMENTS DISCUSSION AND ANA...","(MICROSOFT CORP, [7 MANAGEMENTS DISCUSSIO...","(WALMART INC, None)","(GENERAL ELECTRIC CO, None)"
2,"(Microsoft Corp., [7 MANAGEMENT’S DISCUSSION A...","(Microsoft Corp., [7 MANAGEMENTS DISCUSSION A...","(Microsoft Corp., [7 MANAGEMENTS DISCUSSION A...","(Microsoft Corp., [7 MANAGEMENTS DISCUSSION A...","(Alphabet Inc. Class A, [7MANAGEMENT’S DISCUSS...","(MICROSOFT CORP, [7 MANAGEMENTS DISCUSSION AN...","(MICROSOFT CORP, [7 MANAGEMENTS DISCUSSION AN...","(MICROSOFT CORP, [7 MANAGEMENTS DISCUSSION AN...","(WALMART INC, None)","(PROCTER & GAMBLE CO, None)","(MICROSOFT CORP, [7 MANAGEMENTS DISCUSSION..."


In [25]:
dfTop10_MDA.to_pickle('Top10_MDA.pkl')

The following blocks of code pulls MD&A sections from companies that file the MD&A disclosures in a seperate exhibit 

Get years and SEC web links for scrape.

In [26]:
IBMyears = [i for i in range(2009,2013)]
IBMresLst = ['https://www.sec.gov/Archives/edgar/data/51143/000104746910001151/a2195966zex-13.htm','https://www.sec.gov/Archives/edgar/data/51143/000104746911001117/a2201254zex-13.htm','https://www.sec.gov/Archives/edgar/data/51143/000104746912001742/a2206744zex-13.htm','https://www.sec.gov/Archives/edgar/data/51143/000104746913001698/a2212340zex-13.htm']

WFyears = [i for i in range(2013,2016)]
WFresLst = ['https://www.sec.gov/Archives/edgar/data/72971/000007297114000337/wfc10k_20131231-ex13.htm','https://www.sec.gov/Archives/edgar/data/72971/000007297115000449/wfc-12312014xex13.htm','https://www.sec.gov/Archives/edgar/data/72971/000007297115000449/wfc-12312014xex13.htm']

JJyears = [i for i in range(2007,2015)]
JJyears.remove(2011)
JJyears.remove(2010)
JJresLst = ['https://www.sec.gov/Archives/edgar/data/200406/000095012308002130/y47910exv13.htm','https://www.sec.gov/Archives/edgar/data/200406/000095012309003187/y74152exv13.htm','https://www.sec.gov/Archives/edgar/data/200406/000095012310019392/y80744exv13.htm','https://www.sec.gov/Archives/edgar/data/200406/000020040615000004/ex13-form10xk20141228.htm']

WMyears = [i for i in range(2007,2015)]
WMresLst = ['https://www.sec.gov/Archives/edgar/data/104169/000119312508071085/dex13.htm','https://www.sec.gov/Archives/edgar/data/104169/000010416909000006/dex13.htm','https://www.sec.gov/Archives/edgar/data/104169/000119312510071652/dex13.htm','https://www.sec.gov/Archives/edgar/data/104169/000119312511083157/dex13.htm','https://www.sec.gov/Archives/edgar/data/104169/000119312512134679/d270972dex13.htm','https://www.sec.gov/Archives/edgar/data/200406/000020040613000038/ex13-form10xk20121230.htm','https://www.sec.gov/Archives/edgar/data/200406/000020040614000033/ex13-form10xk20131229.htm','https://www.sec.gov/Archives/edgar/data/104169/000010416915000011/wmtars-1312015.htm']

GEyears = [2007]
GEresLst = ['https://www.sec.gov/Archives/edgar/data/40545/000004054508000011/ex13.htm']

PGyears = [i for i in range(2007,2012)]
PGresLst = ['https://www.sec.gov/Archives/edgar/data/80424/000095015207007152/l27490aexv13.htm#303','https://www.sec.gov/Archives/edgar/data/80424/000095015208007680/l32996bexv13.htm','https://www.sec.gov/Archives/edgar/data/80424/000119312509175986/dex13.htm','https://www.sec.gov/Archives/edgar/data/80424/000119312510188769/dex13.htm','https://www.sec.gov/Archives/edgar/data/80424/000008042411000014/exhibit13.htm']

Pyears = [2008, 2012]
PresLst = ['https://www.sec.gov/Archives/edgar/data/78003/000119312509040568/dex13.htm','https://www.sec.gov/Archives/edgar/data/78003/000007800313000009/pfe-12312012xex1310ka.htm']

ATTyears = [2007, 2008, 2009, 2012]
ATTresLst = ['https://www.sec.gov/Archives/edgar/data/732717/000073271708000012/ex13.htm','https://www.sec.gov/Archives/edgar/data/732717/000073271709000007/ex13.htm','https://www.sec.gov/Archives/edgar/data/732717/000073271710000013/ex13.htm','https://www.sec.gov/Archives/edgar/data/732717/000073271712000025/ex13.htm']

Use pullMDA function to web scrape the SEC website for MDA section in the companies 10-K. Refer to CleanMDA.py for documentation. 

In [27]:
ATTtext = pullMDA(ATTyears, 'AT&T CORP', ATTresLst)
IBMtext = pullMDA(IBMyears, 'INTL BUSINESS MACHINES CORP', IBMresLst)
# Pfizer
Ptext = pullMDA(Pyears, 'PFIZER INC', PresLst)
#Proctor and Gamble 
PGtext = pullMDA(PGyears, 'PROCTER & GAMBLE CO',PGresLst, 'The purpose of this discussion is to provide an understanding')
# GE
GEtext = pullMDA(GEyears,'GENERAL ELECTRIC CO',GEresLst)
# Wal-Mart
WMtext = pullMDA(WMyears,'WALMART INC',WMresLst)
# Johnson & Johnson
JJtext = pullMDA(JJyears,'JOHNSON & JOHNSON',JJresLst)
# Wells Fargo 
WFtext = pullMDA(WFyears,'Wells Fargo',WFresLst)

In [28]:
# Combine all companies into dict, dd.
dd = defaultdict(list)
for d in (GEtext, WMtext,JJtext,WFtext,ATTtext,PGtext,Ptext,IBMtext): 
    for key, value in d.items():
        dd[key].append(value)

In [29]:
# dfScrape is dataframe of scraped companies by year.
dfScrape = pd.DataFrame.from_dict(dd,orient='index')
dfScrape = dfScrape.transpose()

In [30]:
# Read in top 10 from initial web scrape 
top10_MDA = pd.read_pickle('Top10_MDA_Scrape_1.pkl')

In [31]:
# Merge initial scrape to new scrape
frames = [top10_MDA, dfScrape]
top10Mda_Scrape_2 = pd.concat(frames)

In [32]:
top10Mda_Scrape_2.reset_index(inplace=True)

In [38]:
top10Mda_Scrape_2.to_pickle('topMda10_Scrape_2.pkl')

In [39]:
top10Mda_Scrape_2 = top10Mda_Scrape_2.drop(labels='index',axis=1)

In [40]:
# split the df into top 10 companies and the company's to replace their missing values
top10 = top10Mda_Scrape_2[:10]
missing = top10Mda_Scrape_2[10:]
# reformat name Wal-Mart
missing[2013].iloc[0] = 'Wal-Mart Stores Inc',missing[2013].iloc[0][1]
missing[2014].iloc[0] = 'Wal-Mart Stores Inc',missing[2014].iloc[0][1]
# replace the None of top 10 companies with MDA text
for year in top10:
    for i in range(len(top10)):
        if top10[year][i][1] == None:
            for j in range(10,(10+len(missing))):
                if missing[year][j] == None:
                    break
                elif top10[year][i][0].upper() == missing[year][j][0].upper():
                    top10[year].iloc[i] = missing[year][j]
                    break
            

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as ap

In [42]:
# final dataframe with top 10 companies from 2007-2017 with MDA text
top10

,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
0,"(EXXON MOBIL CORP, [15. EXHIBITS, FINANCIAL...","(EXXON MOBIL CORP, [15. EXHIBITS, FINANCIAL...","(EXXON MOBIL CORP, [15. EXHIBITS, FINANCIAL...","(EXXON MOBIL CORP, [15. EXHIBITS, FINANCIAL...","(EXXON MOBIL CORP, [15. EXHIBITS, FINANCIAL ST...","(APPLE INC, [7 MANAGEMENTS DISCUSSION AND AN...","(Apple Inc., [7 MANAGEMENTS DISCUSSION AND A...","(Apple Inc., [7 MANAGEMENTS DISCUSSION AND AN...","(Apple Inc., [7 MANAGEMENTS DISCUSSION AND AN...","(Apple Inc., [7 MANAGEMENT’S DISCUSSION AND AN...","(Apple Inc., [7 MANAGEMENT’S DISCUSSION AND AN..."
1,"(GENERAL ELECTRIC CO, \nEX-13\n4\nex13.htm\n\n...","(WALMART INC, \nEX-13\n3\ndex13.htm\nANNUAL RE...","(MICROSOFT CORP, [7 MANAGEMENTS DISCUSSIO...","(APPLE INC, [7 MANAGEMENTS DISCUSSION AND ANA...","(APPLE INC, [7 MANAGEMENTS DISCUSSION AND ANA...","(EXXON MOBIL CORP, [15. EXHIBITS, FINANCIA...","(Exxon Mobil Corp., [15. EXHIBITS, FINANCI...","(Exxon Mobil Corp., [15. EXHIBITS, FINANCI...","(Alphabet Inc. Class A, [7MANAGEMENT’S DISCUSS...","(Alphabet Inc. Class A, [7MANAGEMENT’S DISCUSS...","(Alphabet Inc. Class A, [7MANAGEMENT’S DISCUSS..."
2,"(MICROSOFT CORP, [7 MANAGEMENTS DISCUSSION...","(PROCTER & GAMBLE CO, of P&G’s financial\nres...","(WALMART INC, \nEX-13\n3\ndex13.htm\nANNUAL RE...","(MICROSOFT CORP, [7 MANAGEMENTS DISCUSSION AN...","(MICROSOFT CORP, [7 MANAGEMENTS DISCUSSION AN...","(MICROSOFT CORP, [7 MANAGEMENTS DISCUSSION AN...","(Alphabet Inc. Class A, [7MANAGEMENT’S DISCUSS...","(Microsoft Corp., [7 MANAGEMENTS DISCUSSION A...","(Microsoft Corp., [7 MANAGEMENTS DISCUSSION A...","(Microsoft Corp., [7 MANAGEMENTS DISCUSSION A...","(Microsoft Corp., [7 MANAGEMENT’S DISCUSSION A..."
3,"(AT&T CORP, \nEX-13\n44\nex13.htm\nAT&AMPT 200...","(MICROSOFT CORP, [7 MANAGEMENTS DISCUSSIO...","(ALPHABET INC, [7 MANAGEMENTS DISCUSSION AND ...","(BERKSHIRE HATHAWAY, [ 7 MANAGEMENTS DISCUSSI...","(INTL BUSINESS MACHINES CORP, \nEX-13\n7\na219...","(GENERAL ELECTRIC CO, [7 “MANAGEMENT’S DISCUSS...","(Microsoft Corp., [7 MANAGEMENTS DISCUSSION A...","(Berkshire Hathaway Inc, [7 MANAGEMENTS DISCU...","(Berkshire Hathaway Inc, [7 MANAGEMENTS DISCU...","(Berkshire Hathaway Inc, [7 MANAGEMENTS DISCU...","(Amazon.com Inc., [7MANAGEMENT’S DISCUSSION AN..."
4,"(PROCTER & GAMBLE CO, of P&G’s financial\nres...","(AT&T CORP, \nEX-13\n44\nex13.htm\nAT&AMPT 200...","(APPLE INC, [7 MANAGEMENTS DISCUSSION AND ANA...","(GENERAL ELECTRIC CO, [7 MANAGEMENTS DISCUSS...","(CHEVRON CORP, [15. EXHIBITS, ...","(INTL BUSINESS MACHINES CORP, \nEX-13\n7\na219...","(Berkshire Hathaway Inc, [7 MANAGEMENTS DISCU...","(Alphabet Inc. Class A, [7MANAGEMENT’S DISCUSS...","(Exxon Mobil Corp., [15. EXHIBITS, FINANCI...","(Exxon Mobil Corp., [15. EXHIBITS, FINANCI...","(Facebook Inc., [7MANAGEMENT'S DISCUSSION AND ..."
5,"(ALPHABET INC, [7 MANAGEMENTS DISCUSSION AND ...","(JOHNSON & JOHNSON, \nEX-13\n4\ny47910exv13.ht...","(JOHNSON & JOHNSON, \nEX-13\n4\ny47910exv13.ht...","(WALMART INC, \nEX-13\n3\ndex13.htm\nANNUAL RE...","(ALPHABET INC, [7 MANAGEMENTS DISCUSSION AND ...","(CHEVRON CORP, [15. EXHIBITS, FINANCIAL STATEM...","(General Electric, [7 “MANAGEMENT’S DISCUSSION...","(JOHNSON & JOHNSON, \nEX-13\n4\ny47910exv13.ht...","(Amazon.com Inc., [7MANAGEMENT’S DISCUSSION AN...","(Amazon.com Inc., [7MANAGEMENT’S DISCUSSION AN...","(Berkshire Hathaway Inc, [7 MANAGEMENTS DISC..."
6,"(CHEVRON CORP, [15. EXHIBITS, ...","(GENERAL ELECTRIC CO, [7 MANAGEMENT’S DISCUSSI...","(PROCTER & GAMBLE CO, of P&G’s financial\nres...","(ALPHABET INC, [7 MANAGEMENTS DISCUSSION AND ...","(WALMART INC, \nEX-13\n3\ndex13.htm\nANNUAL RE...","(JOHNSON & JOHNSON, \nEX-13\n4\ny47910exv13.ht...","(JOHNSON & JOHNSON, \nEX-13\n4\ny47910exv13.ht...","(Wells Fargo, \nEX-13\n6\nwfc10k_20131231-ex13...","(Facebook Inc., [7MANAGEMENT'S DISCUSSION AND ...","(Facebook Inc., [7MANAGEMENT'S DISCUSSION AND ...","(Johnson & Johnson, [7MANAGEMENT’S DISCUSSION ..."
7,"(J

In [43]:
top10.to_pickle('Top10_final.pkl')